In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import sys
from pathlib import Path
import statistics

try:
    sys.path.append(str(Path().cwd().parent))
except IndexError:
    pass

from cqp.bitrate_model.util import *
from cqp.bitrate_model.model import *
from cqp.preprocessor_model.application import *
from cqp.preprocessor_model.model import *

In [ ]:
data_files = list(Path.cwd().joinpath("notebooks").joinpath("data").glob("*.csv"))

max_parameter = MaxBitrateParameter()
dfw = DataFrameWrapper(read_df_by_keys(data_files, KEYS.TEST_SET), max_parameter)

In [ ]:
dfw_train = DataFrameWrapper(read_df_by_keys(data_files, KEYS.TRAINING_SET), max_parameter)
dfw_val = DataFrameWrapper(read_df_by_keys(data_files, KEYS.VALIDATION_SET), max_parameter)

In [ ]:
K_SIZE = list(range(1, 11, 2))
SIGMA = [0.5 * x for x in range(1, 7)]
QP = list(range(24, 46))

analytic_model = BitrateModel(ModelParameter())
analytic_filter_model = AnalyticPreprocessorModel(K_SIZE, SIGMA, analytic_model)

MODEL_PATH = "cqp/models/x264-cif.h5"
model = tf.keras.models.load_model(MODEL_PATH)
model.summary()
ml_filter_model = MLPreprocessorModel.from_dfw(K_SIZE, SIGMA, dfw_train, dfw_val, model)

In [ ]:
qps = get_random_qps(QP, KEYS.TEST_SET_CIF)

qp_single = min(qps.values())
df_single = get_single_encoder_solutions(dfw, qp_single)

In [ ]:
# Reference ground truth for multiple encoders
multi = get_multi_encoder_gt(dfw, qps)
multi

In [ ]:
# Single ground truth for single encoder without preprocessing and min QP
single = get_single_encoder_gt_min_qp(dfw, qps)
single

In [ ]:
sfv_bitrate = multi[KEYS.BITRATE].sum()

# Single ground truth for single encoder without preprocessing and matching BR
single_br = get_single_encoder_gt_matching_br(dfw, qps, sfv_bitrate)
single_br

In [ ]:
# Oracle ground truth for single encoder
oracle = get_oracle_gt(df_single, multi, qps)
oracle

In [ ]:
analytic = pd.DataFrame([analytic_filter_model.estimate_filter_parameter(qp_single, multi.loc[key]) for key in KEYS.TEST_SET_CIF])
analytic["name"] = KEYS.TEST_SET_CIF
analytic

In [ ]:
# Find actual bitrates with filter based on filter model parameter estimations
analytic_gt = get_filter_model_gt(df_single, analytic)
analytic_gt

In [ ]:
# Estimate bitrates for multi encoder parameter and single encoder with filter
ml = pd.DataFrame([ml_filter_model.estimate_filter_parameter(qp_single, multi.loc[key]) for key in KEYS.TEST_SET_CIF])
ml["name"] = KEYS.TEST_SET_CIF
ml

In [ ]:
# Find actual bitrates with filter based on filter model parameter estimations
ml_gt = get_filter_model_gt(df_single, ml)
ml_gt

In [ ]:
single_error_individual = {key: abs(single.loc[key][KEYS.BITRATE].item() - multi.loc[key][KEYS.BITRATE].item()) / dfw.df.loc[key][KEYS.BITRATE].max() for key in KEYS.TEST_SET_CIF}
single_br_error_individual = {key: abs(single_br.loc[key][KEYS.BITRATE].item() - multi.loc[key][KEYS.BITRATE].item()) / dfw.df.loc[key][KEYS.BITRATE].max() for key in KEYS.TEST_SET_CIF}
oracle_error_individual = {key: abs(oracle.loc[oracle["name"] == key][KEYS.BITRATE].item() - multi.loc[key][KEYS.BITRATE].item()) / dfw.df.loc[key][KEYS.BITRATE].max() for key in KEYS.TEST_SET_CIF}
analytic_error_individual = {key: abs(analytic_gt.loc[analytic_gt["name"] == key][f"{KEYS.BITRATE}_x"].item() - multi.loc[key][KEYS.BITRATE].item()) / dfw.df.loc[key][KEYS.BITRATE].max() for key in KEYS.TEST_SET_CIF}
ml_error_individual = {key: abs(ml_gt.loc[ml_gt["name"] == key][f"{KEYS.BITRATE}_x"].item() - multi.loc[key][KEYS.BITRATE].item()) / dfw.df.loc[key][KEYS.BITRATE].max() for key in KEYS.TEST_SET_CIF}

print(f"\nSingle MAE: {statistics.mean(single_error_individual.values())}")
print(f"Single RMSE: {get_rmse_from_abs_difference(single_error_individual.values())}")
print(single_error_individual)
print(f"\nSingle BR MAE: {statistics.mean(single_br_error_individual.values())}")
print(f"Single BR RMSE: {get_rmse_from_abs_difference(single_br_error_individual.values())}")
print(single_br_error_individual)
print(f"\nOracle MAE: {statistics.mean(oracle_error_individual.values())}")
print(f"Oracle RMSE: {get_rmse_from_abs_difference(oracle_error_individual.values())}")
print(oracle_error_individual)
print(f"\nAnalytic MAE: {statistics.mean(analytic_error_individual.values())}")
print(f"Analytic RMSE: {get_rmse_from_abs_difference(analytic_error_individual.values())}")
print(analytic_error_individual)
print(f"\nML MAE: {statistics.mean(ml_error_individual.values())}")
print(f"ML RMSE: {get_rmse_from_abs_difference(ml_error_individual.values())}")
print(ml_error_individual)

In [ ]:
encoding_parameter = collect_experiment_encoding_parameter(multi, single, single_br, oracle, analytic_gt, ml_gt)
encoding_parameter.to_csv("notebooks/ep.csv")

In [ ]:
single_error_individual = []
single_br_error_individual = []
oracle_error_individual = []
analytic_error_individual = []
ml_error_individual = []
ep_list = []

for _ in tqdm(list(range(100))):
    qps = get_random_qps(QP, KEYS.TEST_SET_CIF)
    qp_single = min(qps.values())
    df_single = get_single_encoder_solutions(dfw, qp_single)
    multi = get_multi_encoder_gt(dfw, qps)
    oracle = get_oracle_gt(df_single, multi, qps)
    analytic = pd.DataFrame([analytic_filter_model.estimate_filter_parameter(qp_single, multi.loc[key]) for key in KEYS.TEST_SET_CIF])
    analytic["name"] = KEYS.TEST_SET_CIF
    analytic_gt = get_filter_model_gt(df_single, analytic)
    ml = pd.DataFrame([ml_filter_model.estimate_filter_parameter(qp_single, multi.loc[key]) for key in KEYS.TEST_SET_CIF])
    ml["name"] = KEYS.TEST_SET_CIF
    ml_gt = get_filter_model_gt(df_single, ml)
    single_error_individual.extend([abs(single.loc[key][KEYS.BITRATE].item() - multi.loc[key][KEYS.BITRATE].item()) / dfw.df.loc[key][KEYS.BITRATE].max() for key in KEYS.TEST_SET_CIF])
    single_br_error_individual.extend([abs(single_br.loc[key][KEYS.BITRATE].item() - multi.loc[key][KEYS.BITRATE].item()) / dfw.df.loc[key][KEYS.BITRATE].max() for key in KEYS.TEST_SET_CIF])
    oracle_error_individual.extend([abs(oracle.loc[oracle["name"] == key][KEYS.BITRATE].item() - multi.loc[key][KEYS.BITRATE].item()) / dfw.df.loc[key][KEYS.BITRATE].max() for key in KEYS.TEST_SET_CIF])
    analytic_error_individual.extend([abs(analytic_gt.loc[analytic_gt["name"] == key][f"{KEYS.BITRATE}_x"].item() - multi.loc[key][KEYS.BITRATE].item()) / dfw.df.loc[key][KEYS.BITRATE].max() for key in KEYS.TEST_SET_CIF])
    ml_error_individual.extend([abs(ml_gt.loc[ml_gt["name"] == key][f"{KEYS.BITRATE}_x"].item() - multi.loc[key][KEYS.BITRATE].item()) / dfw.df.loc[key][KEYS.BITRATE].max() for key in KEYS.TEST_SET_CIF])
    ep_list.append(collect_experiment_encoding_parameter(multi, single, single_br, oracle, analytic_gt, ml_gt))

print(f"\nSingle Error: {statistics.mean(single_error_individual)}")
print(f"Single BR Error: {statistics.mean(single_br_error_individual)}")
print(f"Oracle Error: {statistics.mean(oracle_error_individual)}")
print(f"Analytic Error: {statistics.mean(analytic_error_individual)}")
print(f"ML Error: {statistics.mean(ml_error_individual)}")

print(f"\nSingle Error: {get_rmse_from_abs_difference(single_error_individual)}")
print(f"Single BR Error: {get_rmse_from_abs_difference(single_br_error_individual)}")
print(f"Oracle Error: {get_rmse_from_abs_difference(oracle_error_individual)}")
print(f"Analytic Error: {get_rmse_from_abs_difference(analytic_error_individual)}")
print(f"ML Error: {get_rmse_from_abs_difference(ml_error_individual)}")

pd.concat(ep_list).to_csv("notebooks/encoding-parameters.csv")